In [1]:
pip install atproto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.1/351.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.2
    Uninstalling websockets-14.2:
      Successfully uninstalled websockets-14.2
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [2]:
# From the transformers package, import ViTFeatureExtractor and ViTForImageClassification
from transformers import ViTFeatureExtractor, ViTForImageClassification

# From the PIL package, import Image and Markdown
from PIL import Image

# import requests
import requests

# import torch
import torch

# import matplotlib
import matplotlib.pyplot as plt

# url getter for mpl
import urllib

import numpy as np

# import bluesky api
from atproto import Client

# import colab secrets to store login credentials
from google.colab import userdata

# datetime is necessary for caturday check and logging
import datetime
import zoneinfo

# these imports are to use github apis to do logging, base64 is to parse the json
# import requests # already imported for something else
import base64

In [3]:
# input variables - x is target follows y is num of posts we want to look through, whichever end criteria we reach first
EMBEDDED_PIC = 'app.bsky.embed.images#view'
EMBEDDED_VID = 'app.bsky.embed.video#view'
# FEED_CATURDAY = 'at://did:plc:pmyqirafcp3jqdhrl7crpq7t/app.bsky.feed.generator/aaad4sb7tyvjw' # this one is old idk why it disappeared but it was still working?
FEED_CATURDAY = 'at://did:plc:tenurhgjptubkk5zf5qhi3og/app.bsky.feed.generator/#caturday'
FEED_SIAMESE = 'at://did:plc:jv3qdc5vxujp6taaa7nte35i/app.bsky.feed.generator/aaac6wmikqyhq'
FEED_CATPICS = 'at://did:plc:q6gjnaw2blty4crticxkmujt/app.bsky.feed.generator/cv:cat'
FEED_CATS = 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats'
FEED_TUXEDOCATS = 'at://did:plc:eubjsqnf5edgvcc6zuoyixhw/app.bsky.feed.generator/tuxedo-cats'
FEED_NAME = {FEED_CATURDAY: "'Caturday'", FEED_SIAMESE: "'Siamese Cats'", FEED_CATPICS: "'Cat Pics'", FEED_CATS: "'Cats!'", FEED_TUXEDOCATS: "'Tuxedo Cats'"}
URL_BEGIN = 'https://bsky.app/profile/'
URL_POST = '/post/'
# my did to check against
MY_DID = 'did:plc:ktkc7jfakxzjpooj52ffc6ra'

# query creation constants
SCHEMA = '"bsky"'
TABLE_FOLLOWS = '"follows"'
COL_USERID = 'user_id'
COL_USERHANDLE = 'user_handle'
COL_FOLLOWURI = 'follow_uri'
COL_FOLLOWDATE = 'follow_date'
COL_FOLLOWSYOU = 'follows_you'
DATETIME_NOW = 'NOW()'
END_LINE = ','
END_QUERY = ';'
TABLE_POSTS = '"posts"'
COL_POSTCID = 'post_cid'
COL_SEENTIME = 'seen_time'

CATURDAY_DOW = 'Saturday'
USER_TIMEZONE = "US/Eastern" # you should fill this in with your own timezone here

LINE_BREAK = '\n'
END_LOGGING = '\n____________________\n'

# on colab you can set these userdata properties by clicking the key on the left bar, creating a secret, and giving the colab "notebook access"
BSKY_USERNAME = userdata.get('bsky_username')
BSKY_PASSWORD = userdata.get('bsky_password')
GITHUB_TOKEN = userdata.get('github_token')
GITHUB_REPO = userdata.get('github_user/repo') # this value should be <your github username>/<your repo name>
FILE_PATH = "LOGGING_001.txt"  # Replace with the file path in your repo
BRANCH = "main"  # Replace with your branch name

In [4]:
# vision transformer documentation - https://huggingface.co/docs/transformers/v4.13.0/en/model_doc/vit
# guide to using vision transformer from DataCamp - https://www.youtube.com/watch?v=WQMDZniQgj4

# Load the feature extractor for the vision transformer
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
# Load the pre-trained weights from vision transformer
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

### delete posts in the cache that are too old to matter
```
DELETE FROM "bsky"."posts"
WHERE seen_time < NOW() - INTERVAL '2 DAY';
```

### get the remaining existing post entries from the database with this query
```
SELECT post_cid FROM "bsky"."posts";
```

### copy the post ids into the code block below. Here is replacement text so you can ```ctrl+a del``` it when you're done

```
# paste cached post ids here
cached_posts = """

"""
```

# ⭐⭐⭐⭐⭐⭐⭐⭐ this is what you are looking for

In [5]:
# paste cached post ids here
cached_posts = """
...
"""

In [6]:
# now we can split the raw output into lines and turn them into a hashset
cached_posts = set(cached_posts.strip().split('\n'))
seen_posts = set()

print(f'imported {len(cached_posts)} prior seen posts from "posts" table')
print(f'{len(seen_posts)} is the size of our seen posts set -- it should be 0')

imported 5027 prior seen posts from "posts" table
0 is the size of our seen posts set -- it should be 0


In [7]:
# 281: 'tabby, tabby cat'
# 282: 'tiger cat', 283: 'Persian cat', 284: 'Siamese cat, Siamese', 285: 'Egyptian cat', 286: 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor', 287: 'lynx, catamount', 288: 'leopard, Panthera pardus', 289: 'snow leopard, ounce, Panthera uncia', 290: 'jaguar, panther, Panthera onca, Felis onca', 291: 'lion, king of beasts, Panthera leo', 292: 'tiger, Panthera tigris', 293: 'cheetah, chetah, Acinonyx jubatus',
# 281 to 293
cat_labels = set()
for i in range(281, 294):
  cat_labels.add(i)

# these labels are to remove drawings, memes/reposts, and images with a lot of text respectively
bad_labels = {
917 : 'comic book', 916 : 'web site, website, internet site, site', 921 : 'book jacket, dust cover, dust jacket, dust wrapper'}

def test_bsky_image(url):
  f = urllib.request.urlopen(url)
  image = plt.imread(f, format='jpeg')
  # plt.imshow(image)
  inputs = feature_extractor(images=image, return_tensor="pt")
  pixel_values = inputs["pixel_values"]
  pixel_values = np.array(pixel_values)
  pixel_values = torch.tensor(pixel_values)
  outputs = model(pixel_values)
  logits = outputs.logits
  predicted_class_idx = logits.argmax(-1).item()
  sorted_preds = torch.argsort(logits, descending=True)[0]
  top_predictions = [sorted_preds[i].item() for i in range(50)] # 50 is semi-arbitrary based on our findings from testing pics # could see tuning this down to 40 but can't tell if it would pick up more or less cats
  top_values = [logits[0][pred].item() for pred in top_predictions]
  # print('label predictions', top_predictions)
  # print('values of predictions', top_values)
  found_cat_label = -1
  found_bad_label = -1
  bad_labels_found = []
  cat_score = 0
  for i, pred in enumerate(top_predictions):
    predicted_class = model.config.id2label[pred]
    # print(predicted_class)
    if pred in cat_labels:
      if found_cat_label == -1:
        found_cat_label = i
      cat_score += top_values[i]
    if pred in bad_labels:
      if found_bad_label == -1:
        found_bad_label = i
      bad_labels_found.append(pred)
      bad_labels_found.append(bad_labels[pred])
      cat_score -= top_values[i]
  # print(' ')
  print('    found cat label:', found_cat_label)
  print('    found bad label:', found_bad_label, bad_labels_found)
  would_pass = found_cat_label >= 0 and found_bad_label < 0
  # print('AI cat score: ', cat_score)
  # print('    passed cat test:', would_pass)
  return would_pass

In [8]:
# LOG IN TO THE CLIENT
client = Client()
client.login(BSKY_USERNAME, BSKY_PASSWORD)

ProfileViewDetailed(did='did:plc:ktkc7jfakxzjpooj52ffc6ra', handle='tyrowo.com', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreiepqgg5tlozvv4bw5ficwwixflfprionhnli26hngc3hjn6ygpamu@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreierasgybimbz3pfobqfmay5ufxm7jlqnjeiumdx7dyof6mcss7a4m@jpeg', created_at='2023-08-28T14:18:41.780Z', description='Cat poster. Self-taught professional software developer, exF2P MTG Arena Challenger, ex wr holder speedrunning HotD2_100%, ex streamer, ex melee player. \nGrilling, coding, gaming, magic:tg, tcgs, and my kitty, Ricky 😻\nhe/him', display_name='Tyro, ft Ricky', followers_count=3364, follows_count=6907, indexed_at='2025-02-01T19:55:23.213Z', joined_via_starter_pack=None, labels=[], pinned_post=Main(cid='bafyreihszjgxkh5fmu5je5eldv6

In [9]:
# initiate the string to add to our logger file
following = client.get_profile(actor=BSKY_USERNAME).follows_count
followers = client.get_profile(actor=BSKY_USERNAME).followers_count
cur_timestamp = datetime.datetime.now(zoneinfo.ZoneInfo(USER_TIMEZONE))
# print(cur_timestamp)
logging_text = LINE_BREAK + str(cur_timestamp) + LINE_BREAK + 'FINDING NEW FOLLOWERS' + LINE_BREAK + LINE_BREAK + 'prior follower count: ' + str(followers) + LINE_BREAK + 'previously following: ' + str(following) + LINE_BREAK + LINE_BREAK
print(logging_text)
# logging_stats = {}


2025-02-01 18:20:36.929989-05:00
FINDING NEW FOLLOWERS

prior follower count: 3364
previously following: 6907




In [10]:
def like_post_and_add_user(post):
  user_did = post.author.did
  followed_user = client.follow(user_did).uri
  post_cid = post.cid
  post_uri = post.uri
  liked_post = client.like(uri=post_uri, cid=post_cid).uri
  print(f'      ✓✓✓ ✅ Successfully liked post and followed user: {post.author.handle}')
  return followed_user

In [11]:
def get_post_follow_likers(post_uri, like_count, users_added_query_values, already_added_dids, max_new_followers):
  # need to try opening post, get the list of likers, iterate through following them, add each to the users added
  new_follows_count = 0
  likes_remaining = like_count
  try:
    while likes_remaining > 0:
      if (likes_remaining > 100):
        print(f'        Starting new page of likes. {likes_remaining} likes remaining to check on this post.')
      limit = min(likes_remaining, 100)
      likes_remaining -= limit
      next_page = ''
      response = client.get_likes(uri = post_uri, limit= limit, cursor= next_page)
      likes = response.likes
      next_page = response.cursor

      for like in likes:
        you_follow_them = like.actor.viewer.following
        you_are_followed_by = like.actor.viewer.followed_by
        user_did = like.actor.did
        user_handle = like.actor.handle
        if you_follow_them or you_are_followed_by or user_did == MY_DID or user_did in already_added_dids:
          print(f'        Already seen user. handle: {user_handle}')
          continue
        else:
          # like the user
          follow_uri = client.follow(user_did).uri
          date_added = DATETIME_NOW
          follows_you = 'FALSE'
          users_added_query_values.append(f"    ('{user_did}', '{user_handle}', '{follow_uri}', {date_added}, {follows_you}) ")
          print(f'        Followed post-liker. handle: {user_handle}')
          already_added_dids.add(user_did)
          new_follows_count += 1
          if new_follows_count >= max_new_followers:
            break
      # print(f'from {len(likes)} likes on this post you followed {new_followers_count}, saw {follow_them_count} users you already follow, and saw {followed_by_count} users that already follow you')
  except Exception as e:
    print(f'ERROR: THERE WAS AN ISSUE CHECKING THIS POST FOR LIKES. \n{e}')
  return new_follows_count

In [12]:
def createPostUrl(feed_post):
  url_handle = feed_post.post.author.handle
  url_ending_index = feed_post.post.uri.find('.feed.post/') + 11
  url_ending = feed_post.post.uri[url_ending_index : ]
  return URL_BEGIN + url_handle + URL_POST + url_ending

def follow_more_users(post_count, follows_count, feed):
  if post_count == 0 or follows_count == 0:
    return []
  posts_to_check = post_count
  successful_cat_post_like_count = 4
  max_errors_allowed = 5
  next_page = ''
  new_follow_count_from_posts = 0
  new_follow_count_from_likes = 0
  page_count = 0
  users_followed_query_values = []
  already_added_dids = set()

  logging_posts = 0
  logging_pics = 0
  logging_errors_count = 0
  logging_errors_description = []
  logging_notcat = 0
  logging_cat = 0
  logging_vid = 0
  logging_nomedia = 0
  logging_alreadyfollowed = 0
  logging_mutuals = 0
  logging_myposts = 0
  logging_seenpost = 0
  global logging_text
  logging_text = logging_text + f'Feed {FEED_NAME[feed]}:' + LINE_BREAK

  def log_results():
    global logging_text
    sum_new_follows = new_follow_count_from_posts + new_follow_count_from_likes
    print(f'followed {sum_new_follows} new users. {new_follow_count_from_posts} posters and {new_follow_count_from_likes} likers.')
    logging_text = logging_text + f'  Followed {new_follow_count_from_posts + new_follow_count_from_likes} new user{"s" if sum_new_follows != 1 else ""}{"!" if sum_new_follows > 0 else "."}' + LINE_BREAK
    logging_text = logging_text + f'    Of those, {new_follow_count_from_posts} were posters, and {new_follow_count_from_likes} were from likes.' + LINE_BREAK
    logging_text = logging_text + f'  {logging_posts} posts in total were viewed in this run.' + LINE_BREAK
    logging_text = logging_text + f'  Skipped Posts: {logging_seenpost} posts were previously seen, {logging_alreadyfollowed} were from users already followed, {logging_myposts} were your posts.' + LINE_BREAK
    logging_text = logging_text + f'  Mutuals: {logging_mutuals} posts were from users that follow you, and these posts were liked.' + LINE_BREAK
    logging_text = logging_text + f'  Unprocessed: {logging_nomedia} posts had no media attached, and {logging_vid} posts had videos attached.' + LINE_BREAK
    logging_text = logging_text + f'  Processed: {logging_pics} posts had pics attached: {logging_cat} were identified as cat pics and {logging_notcat} were not cats.' + LINE_BREAK
    logging_text = logging_text + f'  {"No errors were encountered while processing pics." if logging_errors_count == 0 else str(logging_errors_count) + " ERROR(S) ENCOUNTERED PROCESSING PICS FROM THIS FEED"} ' + LINE_BREAK
    if logging_errors_count > 0:
      logging_text = logging_text + ';; \n'.join(logging_errors_description) + LINE_BREAK
    logging_text = logging_text + LINE_BREAK # one last line break to get ready for the next entry


  while posts_to_check > 0:
    print(f'[checking page {page_count} of feed {FEED_NAME[feed]}, {posts_to_check} posts left to check, and have found {new_follow_count_from_posts + new_follow_count_from_likes} new users to follow]')
    page_count += 1
    limit = min(posts_to_check, 100)
    posts_to_check -= limit
    data = client.app.bsky.feed.get_feed({
        'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
        'limit': limit,
        'cursor': next_page
    }, headers={})
    next_page = data.cursor

    for i, f in enumerate(data.feed):
      you_follow_them = f.post.author.viewer.following
      you_are_followed_by = f.post.author.viewer.followed_by
      did = f.post.author.did
      post_cid = f.post.cid

      logging_posts += 1
      if did == MY_DID:
        print(f'{i} - 😎 skipped. This was your own post.')
        logging_myposts += 1
        continue
      elif post_cid in cached_posts or post_cid in seen_posts:
        print(f'{i} - 👀 skipped. Post with cid {post_cid} has already been viewed. Found in {"db cache" if post_cid in cached_posts else "current set"}.')
        logging_seenpost += 1
        continue
      else:
        seen_posts.add(post_cid)
        # TODO: the way I have it if you are following them you never check the photo to see if it's a good one to get the likes from.
        if you_follow_them and you_are_followed_by:
          print(f'{i} 💕 user: {f.post.author.handle} is a mutual follower. Liking this post. {createPostUrl(f)}')
          # this can break if you get rate limited. So far hasn't broken when posts are deleted but should have been wrapped in one just in case
          try:
            liked_post = client.like(uri=f.post.uri, cid=f.post.cid).uri
          except Exception as e:
            logging_errors_count += 1
            print(f'    ✓✗ ‼️ liking post {i} caused an error. {logging_errors_count} errors seen this run.\n{e}')
            logging_errors_description.append(f'{i}. {e}')
            # got rate limited and know that if you hit like 100 errors or so you'll eventually get a timeout and the entire nootebook will be borked.
            if logging_errors_count >= max_errors_allowed:
              print(f'seen more errors ({logging_errors_count}) than the acceptable number of errors ({max_errors_allowed}). terminating run.')
              log_results()
              return users_followed_query_values

          logging_mutuals += 1
        elif did in already_added_dids:
          print(f'{i} ✗ 👀 user: {f.post.author.handle} was already followed in this batch.')
          logging_alreadyfollowed += 1
        elif you_follow_them or you_are_followed_by:
          print(f'{i} ✗ 👀 user: {f.post.author.handle} {"already follows you." if you_are_followed_by else ""}{"is already being followed." if you_follow_them else ""}')
          logging_alreadyfollowed += 1
        elif not f.post.embed or f.post.embed.py_type != EMBEDDED_PIC:
          if f.post.embed.py_type == EMBEDDED_VID:
            print(f'{i} ✗ 🎥 video post: {createPostUrl(f)}')
            logging_vid += 1
          else:
            print(f'{i} ✗ 🔲 no pic for post {i}')
            logging_nomedia += 1
        else:
          print(i, '✓', '📷', f.post.embed.images[0].fullsize)
          print(f'    post: {createPostUrl(f)}')
          logging_pics += 1
          try:
            handle = f.post.author.handle
            print(f'    user: {handle}')
            is_cat = test_bsky_image(f.post.embed.images[0].fullsize)
            if is_cat:
              print(f'    ✓✓ 😺 successfully found cat pic at post {i}. It has {f.post.like_count} likes.')
              new_follow_count_from_posts += 1
              logging_cat += 1
              follow_uri = like_post_and_add_user(f.post)
              date_added = DATETIME_NOW
              follows_you = 'FALSE'
              users_followed_query_values.append(f"    ('{did}', '{handle}', '{follow_uri}', {date_added}, {follows_you}) ")
              already_added_dids.add(did)
              # so we have a cat post. If it is a solid or particularly good cat post it should probably have a lot of likes, and we can go in and follow all those likers
              if f.post.like_count >= successful_cat_post_like_count:
                print(f'      👍🏻 This cat post got {f.post.like_count}, and I would call it successful, so following its likers.')
                likers_added = get_post_follow_likers(f.post.uri, f.post.like_count, users_followed_query_values, already_added_dids, follows_count - (new_follow_count_from_posts + new_follow_count_from_likes))
                print(f'      {"✅" if likers_added > 0 else "0️⃣"} Added {likers_added} users that liked that post.')
                new_follow_count_from_likes += likers_added
              if new_follow_count_from_posts + new_follow_count_from_likes >= follows_count:
                print(f'Successfully followed the desired number of new users! breaking out of loop.')
                break
            else:
              print(f'    ✓✗ ❌ post {i} was not a cat pic')
              logging_notcat += 1
          except Exception as e:
            logging_errors_count += 1
            print(f'    ✓✗ ❓ post {i} image caused an error. {logging_errors_count} errors seen this run.\n{e}')
            print(f'errors ({logging_errors_count}), acceptable number of errors ({max_errors_allowed}).')
            logging_errors_description.append(f'{i}. {e}')
            # got rate limited and know that if you hit like 100 errors or so you'll eventually get a timeout and the entire notebook will be borked.
            if logging_errors_count >= max_errors_allowed:
              print(f'seen more errors ({logging_errors_count}) than the acceptable number of errors ({max_errors_allowed}). terminating run.')
              log_results()
              return users_followed_query_values
  log_results()
  return users_followed_query_values

    # print(data.feed[0].post.embed.images[0].fullsize)

In [13]:
# extracting a separate method
def generate_insert_query(rows_list):
  if not rows_list:
    print('NO CHANGES NECESSARY')
    return
  values = ', \n'.join(rows_list)
  print(f'INSERT INTO {SCHEMA}.{TABLE_FOLLOWS} ({COL_USERID}, {COL_USERHANDLE}, {COL_FOLLOWURI}, {COL_FOLLOWDATE}, {COL_FOLLOWSYOU}) ')
  print(f'VALUES ')
  print(values)
  print(f'RETURNING {COL_USERHANDLE}, {COL_FOLLOWDATE}{END_QUERY}')

In [14]:
cur_timestamp = datetime.datetime.now(zoneinfo.ZoneInfo(USER_TIMEZONE))

dow = cur_timestamp.strftime("%A")
is_caturday = dow == CATURDAY_DOW
query_caturday = []

if is_caturday:
  print("IT'S CATURDAY! Checking the Caturday feed for new followers.")
  query_caturday = follow_more_users(2000, 1669, FEED_CATURDAY)
else:
  print("Sorry, today's not Caturday.")

IT'S CATURDAY! Checking the Caturday feed for new followers.
[checking page 0 of feed 'Caturday', 2000 posts left to check, and have found 0 new users to follow]
0 ✓ 📷 https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:e3zhlqozsgod7eswjzj43iy6/bafkreiccoyxxzn7mqovt7f7o7fr2fq5g7wjeshfslxsuwo3wxbe7i4sffe@jpeg
    post: https://bsky.app/profile/linseae.bsky.social/post/3lh5nbo76wc27
    user: linseae.bsky.social


/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `ViTFeatureExtractor.preprocess` and were ignored: 'return_tensor'
  return self.preprocess(images, **kwargs)


Streaming output truncated to the last 5000 lines.
        Already seen user. handle: eternize1179.bsky.social
        Already seen user. handle: bobcg.bsky.social
        Already seen user. handle: saltlife72.bsky.social
        Already seen user. handle: fredkiss.dev
        Already seen user. handle: stacey-mce.bsky.social
        Already seen user. handle: erikaqueue.bsky.social
        Already seen user. handle: shanebonnell.bsky.social
        Already seen user. handle: johnarchbell.bsky.social
        Already seen user. handle: tuffymanji.bsky.social
        Already seen user. handle: adamsills.me
        Already seen user. handle: zmaskm.bsky.social
        Already seen user. handle: rainb0w86.bsky.social
        Already seen user. handle: stevenerd.bsky.social
        Already seen user. handle: astrosfan2023.bsky.social
        Already seen user. handle: mandyfml.bsky.social
        Already seen user. handle: kiryusmalewife.bsky.social
        Already seen user. handle: mribbi

In [15]:
print(f'adding {len(query_caturday)} new follows from caturday.')

adding 1317 new follows from caturday.


In [16]:
generate_insert_query(query_caturday)

INSERT INTO "bsky"."follows" (user_id, user_handle, follow_uri, follow_date, follows_you) 
VALUES 
    ('did:plc:e3zhlqozsgod7eswjzj43iy6', 'linseae.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lh5no74kgv22', NOW(), FALSE) , 
    ('did:plc:4wn6d5ihth7kty4254tmmzof', 'ethansayshey.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lh5noavruk24', NOW(), FALSE) , 
    ('did:plc:jo76c5llaqlhcasnwz3zai3r', 'abobofalltrades.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lh5noidyha2h', NOW(), FALSE) , 
    ('did:plc:gnps7npkyww3kppx5e2pfrqn', 'mytripletsrock.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lh5nokla6c2c', NOW(), FALSE) , 
    ('did:plc:xwpe7c3l7sibhwiykdo47dlh', 'sians.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lh5novcw7s2c', NOW(), FALSE) , 
    ('did:plc:ul7nysjgjfqlwq5o4gx6abny', 'smexyrique.bsky.social', 'at://did:plc:ktkc7jfakxzj

In [17]:
query_siamese = []
if is_caturday:
  print("IT'S CATURDAY! Not running this feed as much")
  query_siamese = follow_more_users(000, 100, FEED_SIAMESE)
else:
  print("Today's not Caturday. Doing the regular routine")
  query_siamese = follow_more_users(1000, 1669, FEED_SIAMESE)

IT'S CATURDAY! Not running this feed as much


In [18]:
print(f'adding {len(query_siamese)} new follows from siamese cats.')

adding 0 new follows from siamese cats.


In [19]:
generate_insert_query(query_siamese)

NO CHANGES NECESSARY


In [20]:
# going through tuxedo and cats after doing siamese has been proving to be nearly completely unnecessary. So we'll skip these for a while
query_cats = []
# follow_more_users(1000, 300, FEED_CATS)

In [21]:
generate_insert_query(query_cats)

NO CHANGES NECESSARY


In [22]:
# going through tuxedo and cats after doing siamese has been proving to be nearly completely unnecessary. So we'll skip these for a while
query_tuxedo = []
# follow_more_users(1000, 300, FEED_TUXEDOCATS)

In [23]:
for line in query_tuxedo:
  print(line)

In [24]:
# create an insert query for the posts cache, updating the db with our newly seen posts
posts_values = [f"  ('{post}', NOW())" for post in seen_posts]
posts_values = ', \n'.join(posts_values)
print(f'INSERT INTO {SCHEMA}.{TABLE_POSTS} ({COL_POSTCID}, {COL_SEENTIME}) ')
print('VALUES')
print(posts_values)
print(f'RETURNING {COL_POSTCID}, {COL_SEENTIME}' + END_QUERY)

INSERT INTO "bsky"."posts" (post_cid, seen_time) 
VALUES
  ('bafyreiecg2ltnjk2ipnisytwayrnsfryphwm3bps6ftncza5r5xaxun5fm', NOW()), 
  ('bafyreifovwkvg7ocxaufreimrcvyindl23hfix7sunawgmgeuawszqzg2a', NOW()), 
  ('bafyreidpurw6sqn54eg4cr76236ipoqg2oxzds24jf7ako5a3gnmonavve', NOW()), 
  ('bafyreidhxuoubhvmdtjc4vnc2geyewgevvimx76os64muj2w6ezz2j6o5u', NOW()), 
  ('bafyreiepshbj2kqrx32hrcdkd6gmb4qdfv6ao5jql3fmksckexvj53xyk4', NOW()), 
  ('bafyreifscpglqvuatnkway7oqfyjl3ocdowg5uhsgewwjirtwd7suoyu6a', NOW()), 
  ('bafyreidecxr7omjxb5uiis2u6f7gxgg3l6fnvfj6tscf7bitmk7i3upcwi', NOW()), 
  ('bafyreidhnoldbuehocnqgozilatp2biamamnjkc5bvyfpugxuxuq77t73a', NOW()), 
  ('bafyreidvq3x5w47lc7ev7i6mfopzgpnpnd45gf2ootaq7lli5p7wcirl6e', NOW()), 
  ('bafyreidnstme5zggxp3a6qvskx6lpvswu5qe3263pxv2mogbedkigzmf3i', NOW()), 
  ('bafyreihq6332ysuw2wchfq6s6aj5ucuoy3lni5bwwuuk65qv5i335ypeti', NOW()), 
  ('bafyreibwzjnkra76q2cx6ss62tiizxlanyi6isncldkw5b337iwa7766zq', NOW()), 
  ('bafyreigijdc3g6fnv5teaa3kmgl7o7eym4nzgp

In [25]:
following = client.get_profile(actor=BSKY_USERNAME).follows_count
followers = client.get_profile(actor=BSKY_USERNAME).followers_count

logging_text = logging_text + LINE_BREAK + 'cur followers: ' + str(followers) + LINE_BREAK + 'now following: ' + str(following) + LINE_BREAK
# print(logging_text)

In [26]:
# LOGGING ALL THE CHANGES TO OUR LOGGING FILE IN GITHUB
commit_message = "Logging for follower additions on " + str(cur_timestamp)

# Step 1: Get the file's current content and SHA
url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{FILE_PATH}"
headers = {"Authorization": f"token {GITHUB_TOKEN}"}
response = requests.get(url, headers=headers)
response_json = response.json()

# Decode the content of the file
file_sha = response_json["sha"]
content = base64.b64decode(response_json["content"]).decode("utf-8")

# Step 2: Modify the file content
new_content = content + LINE_BREAK + logging_text + END_LOGGING
encoded_content = base64.b64encode(new_content.encode("utf-8")).decode("utf-8")

# Step 3: Push the updated content
data = {
    "message": commit_message,
    "content": encoded_content,
    "sha": file_sha,
    "branch": BRANCH,
}
update_response = requests.put(url, headers=headers, json=data)

if update_response.status_code == 200:
    print("Logging file updated successfully! Here's what was added to the logs:\n")
    print(logging_text + END_LOGGING)
else:
    print(f"Error: {update_response.json()}")

Logging file updated successfully! Here's what was added to the logs:


2025-02-01 18:20:36.929989-05:00
FINDING NEW FOLLOWERS

prior follower count: 3364
previously following: 6907

Feed 'Caturday':
  Followed 1322 new users!
    Of those, 317 were posters, and 1005 were from likes.
  762 posts in total were viewed in this run.
  Skipped Posts: 0 posts were previously seen, 45 were from users already followed, 2 were your posts.
  Mutuals: 32 posts were from users that follow you, and these posts were liked.
  Unprocessed: 7 posts had no media attached, and 44 posts had videos attached.
  Processed: 632 posts had pics attached: 317 were identified as cat pics and 315 were not cats.
  5 ERROR(S) ENCOUNTERED PROCESSING PICS FROM THIS FEED 
53. Response(success=False, status_code=429, content=XrpcError(error='RateLimitExceeded', message='Rate Limit Exceeded'), headers={'x-powered-by': 'Express', 'access-control-allow-origin': '*', 'cache-control': 'private', 'vary': 'Authorization, Accep